# Windowing and the discrete Fourier transform
Whenever a discrete Fourier transform is being computed on practical measured data, a finite set of samples are used.  Mathematically, the potentially infinte set of samples has been multiplied by a rectangular sampling function, or *windowed* by the rectangular function.  Unfortunately, because of the duality relationship between multiplication in the time domain and convolution in the frequency domain, as we will see, the resulting transform is affected by *leakage*.

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### User specified parameters
The following parameters can be specified.  

Parameter | Meaning
--------- | -------
<code>N</code> | The number of DFT samples being considered (e.g. 128)
<code>N_high_res </code> | The length of data when zero padding or using longer input sequences, which should be larger than <code>N</code> (e.g. 512)
<code>fft_points</code> | The size of the FFT used to generate continuous plots.  This should be large to obtain a good approximation of the discrete-time Fourier transform (e.g. 4096)
<code>x_lims</code> | Frequency axis limits (Note we want to examine only the central portion of the plot)
<code>y_lims</code> | Magnitude axis limits
<code>xtics</code> | Labels for the frequency axis

In [ ]:
# The number of DFT samples being considered
N = 128
N_high_res = 512

# The size of the FFT used to generate plots
fft_points = 4096

# Graph limits
x_lims = [-51*np.pi/N, 51*np.pi/N]
y_lims = [-40, 40]

# The tick of x axis
xticks = [np.linspace(-3*np.pi/8, 3*np.pi/8, 7),
         ['$-3\pi/8$', '$-\pi/4$', '$-\pi/8$', '0', '$\pi/8$', '$\pi/4$', '$3\pi/8$']]

Based on these parameters, compute time and frequency scales that we will require

In [ ]:
# Scales
time_vals = np.arange(0, N)
time_vals_high_res = np.arange(0, N_high_res)
freq_vals = np.arange(0, fft_points)*2*np.pi/fft_points - np.pi

# DTFT sample points to obtain DFT samples
freq_samples_index = np.arange(0, N) * fft_points / N

### Plotting function
For the majority of plots, the approximation of the discrete-time Fourier transform (DTFT) is plotted, along with the samples obtained by applying the DFT.

In [ ]:
def plot_figure(freq_vals, transform, samples_index, DFT_vals, name, xlim, ylim, xticks):
    """
       Create the plot of the DTFT and DFT samples.  Arguments are:
           freq_vals: DTFT approximation frequencies
           transform: magnitude of DTFT at these frequencies
           samples_index: index into freq_vals of the DFT sample points
           DFT_vals: DFT transform
           name: name to specify the plot filename if producing pdf plots
           xlim: frequency scale limits
           ylim: magnitude scale limits
           xtics: frequency axis values marked on figure
    """
    # Create the plot figure and update label font size
    plt.figure(figsize = (16, 8))
    plt.rcParams.update({'font.size': 16})
    
    # Plot the magnitude of the DTFT approximation
    plt.plot(freq_vals, transform, linewidth=0.8)
    
    # Stem plot of samples from the DTFT approximation
    (markerLines, stemLines, baseLines) = plt.stem(freq_vals[samples_index.astype(int)],
                                                   transform[samples_index.astype(int)], 
                                                   markerfmt = 'ro',
                                                   use_line_collection = True,
                                                   bottom = y_lims[0])
    
    plt.setp(stemLines, color = 'red', linewidth=1) 
    plt.setp(baseLines, color = 'black', linewidth=1) 
    markerLines.set_markersize(4)
    markerLines.set_markerfacecolor('none')

    # Tidy up the plot to control axes sizes and labels
    plt.ylim(ylim)
    plt.xlim(xlim)            
    plt.xticks(xticks[0], xticks[1])
    plt.xlabel('Frequency (radians/sampling interval)')
    plt.ylabel('Magnitude (dB)')
    
    # Save figure in python or ipython system
    if not is_jupyter(): plt.savefig('%s.pdf'%name)

For use later on, we also define a function to draw three lines that generates an arrow given the set of points

In [ ]:
def plot_arrow(start, end, head_size):
    plt.plot([start[0], end[0]], [start[1], end[1]],
            color = 'black', linewidth=0.8)
    plt.plot([end[0]-head_size[0], end[0]],
             [end[1]-head_size[1], end[1]],
            color = 'black', linewidth=0.8)
    plt.plot([end[0]-head_size[2], end[0]],
             [end[1]-head_size[3], end[1]],
            color = 'black', linewidth=0.8)

### Generate figure for windowed cosine example at end of Module 1
Here we generate the plot that was used at the end of module 1, under the heading of windowing.  We are modulating a rectangular function (the input) with a cosine wave.  This is identical to modulating the rectangular function with two phasors and summing them together.
Below we define the signals (constructed as phasors, and also as the cosine itself), and find approximations to their DTFT.

In [ ]:
# Define the inputs for two phasors rotating in opposite directions
input_1 = np.exp(-1j*4*np.pi*time_vals/N)/2
input_2 = np.exp(1j*4*np.pi*time_vals/N)/2
# And define their summation, which is a cosine function
input_sum = input_1+input_2

# Compute an approximation of the DTFT for each signal
transform_1 = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(input_1, fft_points))))
transform_2 = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(input_2, fft_points))))
transform_sum = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(input_sum, fft_points))))

Now we plot the result of this

In [ ]:
# Create the plot figure
plt.figure(figsize = (16, 8))
plt.rcParams.update({'font.size': 16})

# Plot the magnitude of the 3 transforms
plt.plot(freq_vals, transform_1, 
         freq_vals, transform_2, 
         freq_vals, transform_sum,
        )
plt.linewidth = 0.8
plt.ylim(y_lims)
plt.xlim([-51*np.pi/N, 51*np.pi/N])

plt.xticks(np.linspace(-3*np.pi/8, 3*np.pi/8, 7), 
          ['$-3\pi/8$', '$-\pi/4$', '$-\pi/8$', '0', '$\pi/8$', '$\pi/4$', '$3\pi/8$'])
plt.xlabel('Frequency (radians/sampling interval)')
plt.ylabel('Magnitude (dB)')

# Save figure in python or ipython system
if not is_jupyter(): plt.savefig('windowing_example.pdf')

### Effect of windowing on DFT
Now we explore the effect of windowing (considering only a fininte number of samples in the time domain) on what we observe in the frequency domain.  First, let's start with a well defined case.  In the example above, the input has a period of $\frac{N}{2}$, so when we compute an $N$ point DFT, then we find that its frequency lies exactly on the 2nd sample in the frequency domain.  It turns out that in this case, for $N=128$, the only non-zero samples are those at $\pm\frac{\pi}{32}$.  In the plot below, we look only at the central part of the transform to see this in more detail.

In [ ]:
# The name of figure when saved 
name = 'windowing_example_sampled'

# Define the signal we are considering
input_3 = np.cos(4*np.pi*time_vals/N)

# Calculate its DTFT using a high resolution FFT
transform_3 = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(input_3, fft_points))))

# Calculate the DFT
DFT = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(input_3, N))))

# Plot the figure
plot_figure(freq_vals, transform_3, freq_samples_index, DFT, name, x_lims, y_lims, xticks)

### Another signal
This time, we define a new signal, which is also a cosine waveform, however the "period" is $\frac{N}{2.1}$, which is not an integer number of samples!  Its frequency is $\frac{21\pi}{640}$, so it doesn't fall on one of our DFT frequency domain samples which are spaced by $\frac{\pi}{64}$ (assuming $N=128$).  The DTFT shape is very similar to the one above (remember the signal has changed, but only by altering its frequency slightly).  However, when it is sampled, there are many more non-zero samples than before.  In fact, *none* of the DFT values are zero!  This effect is called leakage, and unfortunately it occurs in almost all cases when looking at real data.

In [ ]:
# Define new signal to consider
input_4 = np.cos(4.2*np.pi*time_vals/N)

# Calculate its DTFT
transform_4 = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(input_4, fft_points))))

# and the DFT
DFT = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(input_4, N))))

# The name of figure when saved 
name = 'windowing_example_sampled_2'

# Plot the figure
plot_figure(freq_vals, transform_4, freq_samples_index, DFT, name, x_lims, y_lims, xticks)

### Applying windows
Using the same input signal as above (with the frequency of $\frac{21\pi}{640}$), we multiply it, term by term, with a shaped window.  We will start with the Hann window.  After multiplying, it is processed exactly as before.  Now we can see that the DTFT shape has changed dramatically, and as a result, our DFT samples are more concentrated around the frequency terms corresponding to the input.

In [ ]:
# Get the Hann window
window = np.hanning(N)
windowed_signal = np.multiply(input_4,window)

transform_5 = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(windowed_signal, fft_points))))

# Compute the DFT
DFT = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(windowed_signal, N))))

# The name of figure when saved 
name = 'windowing_example_sampled_hann'

# Plot the figure
plot_figure(freq_vals, transform_5, freq_samples_index, DFT, name, x_lims, y_lims, xticks)

#### Another window
Another commonly used window is the Hamming window.  We process this in exactly the same way as for the Hann window.  This time, the frequency terms initially decay more quickly, but then settles to a level and does not reduce appreciably after that point.

In [ ]:
# Get the Hamming window
window = np.hamming(N)
windowed_signal = np.multiply(input_4,window)

transform_6 = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(windowed_signal, fft_points))))

# Compute the DFT
DFT = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(windowed_signal, N))))

# The name of figure when saved
name = 'windowing_example_sampled_hamming'

# Plot the figure
plot_figure(freq_vals, transform_6, freq_samples_index, DFT, name, x_lims, y_lims, xticks)

### Effect of zero padding
It might be thought that the way to solve this problem is to take more samples in the frequency domain when calculating the DFT.  Unfortunately, it isn't quite the solution you might hope for.  To obtain more frequency domain samples, the number of points in the DFT needs to be increased, but that means the input sequence must be longer.  If you have no more data to use, then all that can be done is to append zeros to the input (zero padding), and then use the longer DFT.  As you will see below, it does result in more frequency domain samples, and the shape of the window is more evident, however it doesn't reduce the amount of leakage.  One thing that it does give, though, is a more accurate indication of the frequency of the input as the peak of the transform is more easily seen.

In [ ]:
# DTFT sample points for zero padding
close_spaced_freq_samples_index = np.arange(0, N_high_res)*fft_points/N_high_res

# Compute the DFT
DFT = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(windowed_signal, N_high_res))))

# the name of figure to be saved
name = 'windowing_example_sampled_hamming_zero_padding'

# Plot the figure 
plot_figure(freq_vals, transform_6, close_spaced_freq_samples_index, DFT, name, x_lims, y_lims, xticks)

### Increase resolution
The only way to increase the resolution of the transform (that is to obtain a frequency representation that can better distinguish different frequency terms), is to increase the length of the input sequence.  Here we extend the length of the input sequence, and use a DFT with more points.  It can be seen that leakage is still present, however the width of the window shapes has significantly reduced, resulting in a much clearer representation of the signal.

In [ ]:
input_5 = np.cos(4.2*np.pi*time_vals_high_res/N)
window = np.hamming(N_high_res)
windowed_signal = np.multiply(input_5,window)

transform_7 = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(windowed_signal, fft_points))))

# Compute the DFT
DFT = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(windowed_signal, N_high_res))))

# the name of figure to be saved
name = 'windowing_example_sampled_hamming_higher_resolution'

# Set the range of y axis
ylim = [-40, 45]

# Plot the figure 
plot_figure(freq_vals, transform_7, close_spaced_freq_samples_index, DFT, name, x_lims, ylim, xticks)

### Window parameters
Clearly the window that is used has a strong influence on the resulting transform.  There are some properties that are important for windows, and the plot below shows these.  To display the window, zero padding is applied to the samples of the window before calculating the DFT.

Python supports the following windows:

Window | Function
------ | --------
Bartlett | <code>np.bartlett(length)</code>
Hanning | <code>np.hanning(length)</code>
Hamming | <code>np.hamming(length)</code>
Blackman | <code>np.blackman(length)</code>
Kaiser | <code>np.kaiser(length, beta)</code>

In [ ]:
# Produce plot for window parameters
# Create an array with N ones
input_6 = np.ones(N)
window = np.hamming(N)

# The transform of input
fftshift = np.abs(np.fft.fftshift(np.fft.fft(np.multiply(input_6, window), fft_points)))

# The first element in fftshift is zero which is infinity in log10, 
# substitute it with an infinitisimal number
fftshift[0] = 1e-99
transform_8 = 20*np.log10(np.abs(fftshift))

# Create the plot figure
plt.figure(figsize = (16, 8))
plt.rcParams.update({'font.size': 16})

# Plot the magnitude of the transform_4 in dB scale
plt.plot(freq_vals, transform_8, linewidth=0.8)

# Tidy up the plot to control axes sizes and labels
# Get current axes instance
ax = plt.gca()

peak_value = transform_8[int(fft_points/2)]
plt.ylim([peak_value-60, peak_value+10])
plt.xlim([-np.pi, np.pi])

plt.xticks(np.linspace(-np.pi, np.pi, 5),
          ['$-\pi$', '$-\pi/2$', '0', '$\pi/2$', '$\pi$'])
plt.xlabel('Frequency (radians/sampling interval)')
# Let y axis invisible
ax.get_yaxis().set_visible(False)

# Mark up the main lobe width first
# Vertical bars
plt.plot([-0.027*np.pi, -0.027*np.pi], [peak_value, peak_value-10], 
          color = 'black', linewidth=0.8)
plt.plot([0.027*np.pi, 0.027*np.pi], [peak_value, peak_value-10], 
          color = 'black', linewidth=0.8)

# Arrows
plot_arrow([-0.14*np.pi, peak_value-5],
           [-0.027*np.pi, peak_value-5],
           [0.03*np.pi, 1, 0.03*np.pi, -1])
plot_arrow([0.14*np.pi, peak_value-5],
           [0.027*np.pi, peak_value-5],
           [-0.03*np.pi, 1, -0.03*np.pi, -1])

# Now the sidelobe height
# Horizontal bars
plt.plot([0.1*np.pi, 0.4*np.pi], [peak_value, peak_value], 
          color = 'black', linewidth=0.8)
plt.plot([0.1*np.pi, 0.4*np.pi], [peak_value-42.5, peak_value-42.5], 
          color = 'black', linewidth=0.8)

# Arrow
plot_arrow([0.25*np.pi, peak_value-42.5],
           [0.25*np.pi, peak_value],
           [0.02*np.pi, 3, -0.02*np.pi, 3])
plot_arrow([0.25*np.pi, peak_value],
           [0.25*np.pi, peak_value-42.5],
           [0.02*np.pi, -3, -0.02*np.pi, -3])

# Now add the text annotations
plt.text(-6*np.pi/16, 31, 'Resolution', FontSize = 16)
plt.text(np.pi/4+0.1, 15, 'Peak Sidelobe level', FontSize = 16)

# Save figure in python or ipython system
if not is_jupyter(): plt.savefig('window_parameters.pdf')

The parameters of interest are the peak sidelobe level, and the resolution.
- The peak sidelobe level is defined as the difference between the peak of the main lobe (the peak centred on 0), and the highest peak in the rest of the transform.  This controls the dynamic range of the transform as any signal components that lie below this level may not be able to be seen due to the leakage from the largest signal component.
- The resolution defines the minimum separation of two equal power frequency components that allows the two components to be identified by the transform.  If the frequencies are closer than this, then the transform will not be able to distinguish between them, and they will be treated as a single component.

### The effect of windowing for more complex signals
We will now apply the windowing technique to signals with multiple components so that the effects of limited dynamic range and resolution can be more readily understood.  First, alter some of our previous settings:

In [ ]:
N = 100

# The size of the FFT used to generate the plots so that the
# DFT frequency samples align with the approximated DTFT samples
fft_points = 4000

# Graph limits
y_lims = [-40, 40]
x_lims = [0, 2*np.pi]

# The tick of x axis
xtick = [np.linspace(0,2*np.pi,5), 
        ['0', '$\pi/2$', '$\pi$', '$3\pi/2$', '$2\pi$']]

# Adjust time scale for the different number of samples
time_vals = np.arange(0, N)
# Adjust frequency scale to show from 0 to 2\pi
freq_vals = np.arange(0, fft_points) * 2 * np.pi / fft_points

# DTFT sample points
freq_samples_index = np.arange(0, N) * fft_points / N

### Resolution using an unwindowed transform
Our signal comprises three equally weighted frequency components, two of which are very close in frequency.  Initially we do not apply a shaped window, often (mistakenly) called an unwindowed transform.  However, as we are only dealing with a finite length of the input, we are actually applying a rectangular window implicitly.  The resolution of the rectangular window is 0.89 samples (in the frequency domain), so in the example below, the closely spaced frequencies can be identified as they are separated by 1 sample.

In [ ]:
input_7 = ( np.cos(0.6*np.pi*time_vals) 
          + np.cos(0.2*np.pi*time_vals) 
          + np.cos(0.22*np.pi*time_vals))
transform_9 = 20*np.log10(abs(np.fft.fft(input_7, fft_points)))

# Compute the DFT
DFT = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(input_7, N))))

# The name of figure when saved
name = 'windowing_example_resolution_unwindowed'

# Plot the figure
plot_figure(freq_vals, transform_9, freq_samples_index, DFT, name, x_lims, y_lims, xtick)

### Applying a window
In the case above, the signal components could be resolved (identified) when using the implicit rectangular window.  No other window has as good a resolution, so when we apply the Hamming window, we can see that it is no longer possible to identify the two separate components.  The only solution to this problem is to increase the length of the input, and use a longer window, which will result in a narrower main lobe, but still control the peak sidelobe height.

In [ ]:
window = np.hamming(N)
windowed_signal = np.multiply(input_7, window)
transform_10 = 20*np.log10(abs(np.fft.fft(windowed_signal, fft_points)))

# Compute the DFT
DFT = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(windowed_signal, N))))

# The name of figure when saved
name = 'windowing_example_resolution_hamming'

# Plot the figure
plot_figure(freq_vals, transform_10, freq_samples_index, DFT, name, x_lims, y_lims, xtick)

### Dynamic range
Again, we will start with a case where we do not apply a shaped window (the so called unwindowed case).  This time, the signal consists of two frequency terms, well separated in the frequency domain, however one is 40 dB lower than the other.  In the plot below you will see that only one component, and its leakage caused by the rectangular window, is able to be observed.  The peak sidelobe level of a rectangular window is -13 dB, and the decay after this is quite slow, so it is no surprise that the small signal cannot be observed.

In [ ]:
input_8 = np.cos(0.205*np.pi*time_vals) + 0.01*np.cos(0.4*np.pi*time_vals)
transform_11 = 20*np.log10(abs(np.fft.fft(input_8, fft_points)))

# Compute the DFT
DFT = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(input_8, N))))

# The name of figure when saved
name = 'windowing_example_dynamic_unwindowed'

# Plot the figure
plot_figure(freq_vals, transform_11, freq_samples_index, DFT, name, x_lims, y_lims, xtick)

Taking the same signal, and applying a Hamming window (which has a peak sidelobe level of -43 dB), the lower power component in the input can be identified.

In [ ]:
windowed_signal = np.multiply(input_8, window)
transform_12 = 20*np.log10(abs(np.fft.fft(windowed_signal, fft_points)))

# Compute the DFT
DFT = 20*np.log10(abs(np.fft.fftshift(np.fft.fft(windowed_signal, N))))

# The name of figure when saved
name = 'windowing_example_dynamic_hamming'

# Plot the figure
plot_figure(freq_vals, transform_12, freq_samples_index, DFT, name, x_lims, y_lims, xtick)

### Summary
What these two examples demonstrate is that there is a trade-off to be made between dynamic range, resolution and the length of the input sequence.  For every problem that you come across in practice, you will need to make these design choices.  I would recommend that you start with peak sidelobe level to choose the window that will give you the dynamic range you require, then determine the length of input sequence you will need in order to obtain the resolution you require.

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.